In [1]:
import ee
import geemap
import importlib
import ast
import pandas as pd

In [2]:
Map = geemap.Map()

In [3]:
ee.Initialize()

In [4]:
import utils.ee_utils as ee_utils

In [5]:
# Reload the module
importlib.reload(ee_utils)

<module 'utils.ee_utils' from 'd:\\python\\CropMapping\\dataset\\utils\\ee_utils.py'>

In [6]:
date1 = '2018-8-01'
date2 = '2018-8-30'

In [7]:

# Define area of interest.
# roi = ee.Geometry.Polygon(
#                                 [[[1.2897224677359675, 48.38400739957012],
#                                 [1.2897224677359675, 47.93697958008443],
#                                 [3.2068367255484675, 47.93697958008443],
#                                 [3.2068367255484675, 48.38400739957012]]])
roi = ee.Geometry.Polygon( [[[2.220233, 48.116767],
                             [2.301807, 48.116767],
                             [2.301807, 48.167835],
                             [2.220233, 48.167835],
                             [2.220233, 48.116767]]])


# Call function with defined start and end dates and area of interest.
cloudless_collection = ee_utils.create_s2_collection(roi, date1, date2)

# Define visualization parameters.
visualization = {
        'min': 0.0,
        'max': 0.3,
        'bands': ['B4', 'B3', 'B2'],
}

# Add layer to map.
Map.addLayer(cloudless_collection.median().clip(roi), visualization, 'Cloudless Collection')
Map.centerObject(roi, 10)

In [8]:
s1_mosaic = ee_utils.get_combined_s1_image(roi, date1, date2)
s1_mosaic = ee_utils.sen1_add_ratio(s1_mosaic, vv_name='VV', vh_name='VH', ratio_name='ratio')

false_color = {
    'bands': ['VV', 'VH', 'ratio'],
    'min': [-25, -35, 0],
    'max': [0, -5, 10]
}
Map.addLayer(s1_mosaic, false_color, 'S1 Mosaic')
Map.centerObject(roi, 10)

In [9]:
crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018')
# binari mask for crop map for mazie 216
crop_map = crop_map.eq(216)

# Replace null values with 0
crop_map = crop_map.unmask(0).clip(roi)

Map.addLayer(crop_map, {}, 'Crop Map')

In [10]:
s1_mosaic

In [11]:
Map

Map(center=[48.14230398135283, 2.2610199999944824], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
# get roi from polygon geemap
# roi = Map.draw_last_feature.geometry()
# print(roi)

In [13]:
print(roi.getInfo())

{'type': 'Polygon', 'coordinates': [[[2.220233, 48.116767], [2.301807, 48.116767], [2.301807, 48.167835], [2.220233, 48.167835], [2.220233, 48.116767]]]}


In [14]:
sen1_short_print = lambda s2_collection: ee_utils.ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                           'system:band_names','instrumentMode',                         
                                                                                            'relativeOrbitNumber_stop','relativeOrbitNumber_start',
                                                                                            'cycleNumber'],
                                                             first_is_t_in_millis=True, df_instead_of_print=False)

In [15]:
def download_s1_s2(roi, start_date, end_date, s1_folder="", s2_folder="",
                   s1_file_name="s1_56-48.tif", s2_file_name="s2_56-48.tif",
                   scale=10, bands=['B2', 'B3', 'B4'], proj='EPSG:3857'):
    s1_img = ee_utils.get_combined_s1_image(roi, start_date, end_date)
    s2_img = ee_utils.get_combined_s2_image(roi, start_date, end_date)
    s2_img = s2_img.select(bands)

    # EPSG:4326 or EPSG:3857
    geemap.download_ee_image(s1_img, s1_folder + s1_file_name, crs=proj, scale=scale, region=roi)
    geemap.download_ee_image(s2_img, s2_folder + s2_file_name, crs=proj, scale=scale, region=roi)

In [16]:
# download_s1_s2(roi, '2019-05-01', '2019-06-30', scale=10, bands=['B2','B3','B4'])

In [17]:
# crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018').clip(roi)
# geemap.download_ee_image(crop_map, "crop_map"+".tif",crs='EPSG:3857', scale=10,region = roi)

In [18]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
def get_previous_months(harvest_date):
    # Convert harvest date string to datetime object
    harvest_datetime = datetime.strptime(harvest_date, '%Y-%m-%d')
    
    # Create a list to store the previous months
    previous_months = []
    
    # Loop through the previous six months and add them to the list
    for i in range(1, 7):
        previous_month = ((harvest_datetime - relativedelta(months=i+1)).strftime('%Y-%m-%d'), (harvest_datetime - relativedelta(months=i)).strftime('%Y-%m-%d'))
        
        previous_months.append(previous_month)
        
    previous_months.reverse()
    return previous_months

dummy_harvest_date = '2019-08-01'
previous_months = get_previous_months(dummy_harvest_date)
print(previous_months)

[('2019-01-01', '2019-02-01'), ('2019-02-01', '2019-03-01'), ('2019-03-01', '2019-04-01'), ('2019-04-01', '2019-05-01'), ('2019-05-01', '2019-06-01'), ('2019-06-01', '2019-07-01')]


In [19]:
def name_from_roi(roi):
    center = roi.centroid().getInfo()['coordinates']
    x = str(center[0]).replace('.','-')[0:5]
    y = str(center[1]).replace('.','-')[0:5]
    return x + "_" + y + ".tif"

name_from_roi(roi)

'2-261_48-14.tif'

In [20]:
import os
def check_folder_exists(folder):
    if not os.path.exists(folder):
        print("Creating folder: " + folder)
        os.makedirs(folder)

In [21]:
def ts_downloader(roi, harvest_date, scale=10, bands=['B2','B3','B4'], proj='EPSG:3857', dataset_folder="ts_dataset"):
    ts_dates = get_previous_months(harvest_date)
    name = name_from_roi(roi)
    for i, ts_date in enumerate(ts_dates):
        print(f"Downloading Date {i+1}: {ts_date[0]} to {ts_date[1]}")
        check_folder_exists(dataset_folder + "//s1//" + str(i+1))
        check_folder_exists(dataset_folder + "//s2//" + str(i+1))
        download_s1_s2(roi, ts_date[0], ts_date[1], s1_folder="ts_dataset//s1//" + str(i+1) + "//",
                                                    s2_folder="ts_dataset//s2//" + str(i+1) + "//",
                                                    s1_file_name=name,  s2_file_name=name,
                                                    scale=scale, bands=bands)
    
    print("Downloading Crop Map...")
    crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018').clip(roi)
    # none vlue to 0
    crop_map = crop_map.unmask(0)
    check_folder_exists("ts_dataset//crop_map")
    geemap.download_ee_image(crop_map, dataset_folder + "//" + "crop_map//" + name, crs='EPSG:3857', scale=10,region = roi)
    
    print("Done!")

In [51]:
rois_path = f'D:\\python\\CropMapping\\dataset\\Europe.xlsx'

In [52]:
df = pd.read_excel(rois_path)

In [53]:
BANDS = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

In [56]:
for index, row in df.iterrows():
    print(f"Processing {index+1} of {len(df)}")
    r_roi = row['roi']
    r_roi = r_roi.replace('\n', '').replace(' ', '')
    r_roi = ast.literal_eval(r_roi)
    r_roi = ee.Geometry.Polygon(r_roi)
    r_date = row['harvest_date']
    ts_downloader(r_roi, r_date, scale=10, bands=BANDS)

Processing 1 of 8


0-398_52-49.tif: |          | 0.00/34.9M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


0-398_52-49.tif: |          | 0.00/87.3M (raw) [  0.0%] in 00:00 (eta:     ?)

: 

In [31]:
ts_downloader(roi, dummy_harvest_date, scale=10, bands=['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12'])

Creating folder: ts_dataset//s2//1


2-261_48-14.tif: |          | 0.00/12.4M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


2-261_48-14.tif: |          | 0.00/31.0M (raw) [  0.0%] in 00:00 (eta:     ?)

Creating folder: ts_dataset//s1//2
Creating folder: ts_dataset//s2//2


...

2-261_48-14.tif: |          | 0.00/12.4M (raw) [  0.0%] in 00:00 (eta:     ?)

2-261_48-14.tif: |          | 0.00/31.0M (raw) [  0.0%] in 00:00 (eta:     ?)

Creating folder: ts_dataset//crop_map


2-261_48-14.tif: |          | 0.00/1.55M (raw) [  0.0%] in 00:00 (eta:     ?)

Done!


In [36]:
from utils import utils
importlib.reload(utils)

<module 'utils.utils' from 'd:\\python\\CropMapping\\dataset\\utils\\utils.py'>

In [33]:
os.listdir("ts_dataset//")

['crop_map', 's1', 's2']

In [34]:
# ts_dataset
# |-- crop_map
# |-- s1
#     |-- 1
#     |-- 2
#     |-- 3
#     |-- 4
#     |-- 5
#     |-- 6
# |-- s2
#     |-- 1
#     |-- 2
#     |-- 3
#     |-- 4
#     |-- 5
#     |-- 6

# Loop through the folders in the dataset


In [37]:

main_in_folder = "ts_dataset//"
main_out_folder = "ts_dataset_patched//"
folders = ["s1", "s2"]
for folder in folders:
    ts_date_folders = os.listdir(main_in_folder + folder)
    for date in ts_date_folders:
        print("Processing: " + main_in_folder + folder + "//" + date)
        input_sat = "S2" if folder == "s2" else "S1"
        utils.patch_folder(main_in_folder + folder + "//" + date+ "//", main_out_folder + folder + "//" + date+ "//", input_sat=input_sat, remove_year=False,mute=True)

# patch crop map
print("Processing: " + main_in_folder + "crop_map" + "//")
utils.patch_folder(main_in_folder + "crop_map" + "//", main_out_folder + "crop_map" + "//", input_sat="crop_map", remove_year=False,mute=True)


Processing: ts_dataset//s1//1
 2-261_48-14.tif 
shape after fix: (853, 909, 2)
range before norm:  -49.983915584353966 -13.449824945360952 5.372152874504233 20.771304398343776
NaN Ratio: 0.0 Percent
range after norm and NaN removal:   -25.0 -13.420638424887525 5.294002526284856 15.0 
✅  Final Shape-> (15, 15, 64, 64, 2) 
Processing: ts_dataset//s1//2
 2-261_48-14.tif 
shape after fix: (853, 909, 2)
range before norm:  -50.26328759605357 -15.768953254918932 6.0358281339830775 20.418096725876534
NaN Ratio: 0.0 Percent
range after norm and NaN removal:   -25.0 -15.586981331462495 5.636057983771641 15.0 
✅  Final Shape-> (15, 15, 64, 64, 2) 
Processing: ts_dataset//s1//3
 2-261_48-14.tif 
shape after fix: (853, 909, 2)
range before norm:  -47.46076703464347 -15.260919953144727 5.395244365007187 20.361222064075424
NaN Ratio: 0.0 Percent
range after norm and NaN removal:   -25.0 -15.217241649344631 5.300207042349706 15.0 
✅  Final Shape-> (15, 15, 64, 64, 2) 
Processing: ts_dataset//s1//4
 2